In [1]:
# Import packages
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [2]:
# Setting bearer token as an environment variable
# os.environ['TOKEN'] = '<ADD_BEARER_TOKEN>'

In [4]:
def auth():
  return os.getenv('TOKEN')

In [14]:
# Create headers
def create_headers(bearer_token):
  headers = {"Authorization": "Bearer {}".format(bearer_token)}
  return headers

In [15]:
# Create URL
def create_url(keyword, start_date, end_date, max_results = 10):
  search_url = "https://api.twitter.com/2/tweets/search/recent"

  query_params = {'query': keyword,
                  'start_time': start_date,
                  'end_time': end_date,
                  'max_results': max_results,
                  'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                  'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                  'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                  'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                  'next_token': {}}
  return (search_url, query_params)

In [16]:
# Connect to Endpoint
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   # params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [21]:
# Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "elders"
start_time = "2022-01-12T00:00:00.000Z"
end_time = "2022-01-17T00:00:00.000Z"
max_results = 15

In [22]:
# Creating URL and JSON response object
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [ ]:
# Response data
print(json.dumps(json_response, indent=4, sort_keys=True))

In [31]:
# Tweet response text, dates, and time
for tweet in json_response['data']:
  print(tweet['created_at'], tweet['text'])

2022-01-16T23:59:55.000Z RT @SpiesVespers: IT'S TIME FOR STRANGE SPY FACTS! DEZINFORMATSIYA AND PROPAGANDA EDITION! 
The 1st English translation of Protocol of Elde…
2022-01-16T23:59:36.000Z RT @SeffSaid: "I’m old school. I believe in having good manners, respecting my elders and helping others when I can." - Unknown
2022-01-16T23:59:31.000Z RT @EthernalElves: SeNTinels finally step foot in the Elders Sanctum.  Ethereal reverbs used to echo throughout the Sanctum now filled with…
2022-01-16T23:59:22.000Z @DecentFilms Is this review another complaint about kids being right and elders being wrong? If so…please don’t. 😕
2022-01-16T23:59:06.000Z @revisser @JoostNiemoller Het probleem zit hem niet in de kleine partijtjes, maar juist in regeringspartijen met hun "functie elders", en het gelieg en gedraai daarna. Die daarna elkaar ook nog gaan uitsluiten om elkaar daarna na heel veel gezeik en theater weer in de armen te vallen.
2022-01-16T23:59:01.000Z RT @Ebonyteach: Dr. King was only 39 wh

In [26]:
## Saving tweets to .csv file
# Creating .csv
csvFile = open("data/test_tweets.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

# DF headers
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

def append_to_csv(json_response, fileName):
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    for tweet in json_response['data']:
        
        author_id = tweet['author_id']
        created_at = dateutil.parser.parse(tweet['created_at'])
        
        if ('geo' in tweet):                # Filtering for unspecified geo locations
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        tweet_id = tweet['id']

        lang = tweet['lang']

        # Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # Source
        source = tweet['source']

        # Tweet text
        text = tweet['text']
        
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    csvFile.close()

    # Print the number of tweets for this iteration
    print("# Tweets added from this response: ", counter)

In [27]:
append_to_csv(json_response, "data/test_tweets.csv")

# Tweets added from this response:  15
